In [15]:
# Note: need to run the following in powershell
#       pip install polygon-api-client

import pandas as pd
from polygon import RESTClient

# Note: the furthest back we can go with the free plan seems to be exactly 2 years from the current date
key = '<API_KEY>'
client = RESTClient(key)

In [16]:
from datetime import datetime

# Note:  the API lists timestamps rather than dates.
#        These are unix millisecond timestamps, which describe the number of milliseconds 
#        since Jan 1, 1970, midnight UTC/GMT.
#        We use the following function to convert timestamps to date strings

def timestamp_to_date(timestamp):
    """
    Convert a UNIX timestamp in milliseconds to a date string in the format YYYY-MM-DD.
    """
    
    # Convert to seconds and shift by a day to align with data
    epoch_secs = timestamp/1000 + 86400
    
    # Convert the timestamp to a datetime object
    dt_object = datetime.fromtimestamp(epoch_secs)
    
    # Convert to string of form YYYY-MM-DD
    date_string = dt_object.strftime('%Y-%m-%d')
    
    return date_string

In [26]:
def multiple_stock_table(date):
    """
    Returns a dataframe containing all info on all stocks at a single specified date
    Date is a string of the form YYYY-MM-DD
    Dataframe is sorted by trade volume
    """
    
    data = client.get_grouped_daily_aggs(date)

    df = pd.DataFrame({
        'Close' : [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open' : [stock.open for stock in data],
        'High' : [stock.high for stock in data],
        'Low' : [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
        index = [stock.ticker for stock in data])
    
    df = df.sort_values('Volume', ascending = False)
    
    return df


In [27]:
df = multiple_stock_table('2023-02-14')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
NVDA,22.9710,675435900.0,21.5780,23.0488,21.3660,22.5682,697353.0
TQQQ,25.4900,255737195.0,24.5350,25.7400,24.0500,24.9832,599839.0
TSLA,209.2500,216334008.0,191.9400,209.8200,189.4400,201.8225,1819451.0
PLTR,9.2200,215980237.0,8.5900,9.2950,8.2300,8.7592,458742.0
SRNE,0.1975,190516566.0,0.2490,0.2684,0.1825,0.2158,116035.0
...,...,...,...,...,...,...,...
HIYS,25.0450,0.0,25.0450,25.0450,25.0450,NaN,NaN
SYII,42.5443,0.0,42.5443,42.5443,42.5443,NaN,NaN
AUGZ,31.6146,0.0,31.6146,31.6146,31.6146,NaN,NaN
SUBS,18.6122,0.0,18.6122,18.6122,18.6122,NaN,NaN


In [28]:
def single_stock_table(ticker, start_date, end_date):
    """
    Returns a dataframe containing all info on a given stock from start_date to end_date
    Dates are strings of the form YYYY-MM-DD
    """
    
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    df = pd.DataFrame({
    'Close' : [entry.close for entry in data],
    'Volume': [entry.volume for entry in data],
    'Open' : [entry.open for entry in data],
    'High' : [entry.high for entry in data],
    'Low' : [entry.low for entry in data], 
    'Vwap': [entry.vwap for entry in data],
    'Transactions': [entry.transactions for entry in data]},
    index = [timestamp_to_date(entry.timestamp) for entry in data]
    )
    
    return df



In [33]:
df = single_stock_table('NVDA', '2022-08-01', '2025-02-12')
df

,Close,Volume,Open,High,Low,Vwap,Transactions
2023-02-14,22.971,675435900.0,21.578,23.0488,21.3660,22.5682,697353
2023-02-15,22.764,421148660.0,22.549,22.8550,22.1060,22.5468,471919
2023-02-16,22.002,412026280.0,22.133,22.5500,21.9268,22.2462,473577
2023-02-17,21.388,465887640.0,21.631,21.7400,20.9750,21.3416,552526
2023-02-21,20.655,410014550.0,21.000,21.4938,20.6180,20.9405,427159
...,...,...,...,...,...,...,...
2025-02-06,128.680,248146032.0,127.420,128.7700,125.2100,127.3319,1855949
2025-02-07,129.840,226630821.0,129.220,130.3700,125.0000,129.0634,1778758
2025-02-10,133.570,211358778.0,130.090,135.0000,129.9600,133.6341,1658255
2025-02-11,132.800,175349571.0,132.580,134.4800,131.0200,133.1308,1292554


In [ ]:
import requests 

def fetch_macd(stock_ticker, timespan="day", short_window=12, long_window=26, signal_window=9, limit=10):
    """
    Fetch MACD data from Polygon API for a given stock.

    If the MACD is higher than the signal line there is a upward trend and should think of buying 
    If the MACD is lower than the signal line there is a downwards trend and should think of selling
    """
    url = f"https://api.polygon.io/v1/indicators/macd/{stock_ticker}?timespan={timespan}&adjusted=true&short_window={short_window}&long_window={long_window}&signal_window={signal_window}&series_type=close&order=desc&limit={limit}&apiKey={key}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching MACD data: {response.json()}")

    data = response.json()
    
    if "results" not in data or "values" not in data["results"]:
        raise Exception("No MACD data available")

    macd_values = data["results"]["values"]
    df = pd.DataFrame(macd_values)[["timestamp", "value", "signal"]]
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df.rename(columns={"value": "MACD", "signal": "Signal_Line"}, inplace=True)
    
    return df


macd_df = fetch_macd("AAPL")
macd_df

,timestamp,MACD,Signal_Line
0,2025-02-12 05:00:00,-1.700903,-2.389348
1,2025-02-11 05:00:00,-2.286967,-2.561460
2,2025-02-10 05:00:00,-2.569007,-2.630083
3,2025-02-07 05:00:00,-2.385588,-2.645351
4,2025-02-06 05:00:00,-2.114743,-2.710292
5,2025-02-05 05:00:00,-2.307451,-2.859179
6,2025-02-04 05:00:00,-2.443489,-2.997112
7,2025-02-03 05:00:00,-2.615181,-3.135517
8,2025-01-31 05:00:00,-2.308575,-3.265601
9,2025-01-30 05:00:00,-2.697182,-3.504858


In [37]:
def stock_proc(ticker, start_date, end_date, n):
    """
    Fetch stock data for a given ticker between start_date and end_date,
    and compute the Price Rate of Change (PROC) over the last 'n' days.
    Automatically fixes incorrect date order.
    """
    # Ensure correct date order
    start_date, end_date = min(start_date, end_date), max(start_date, end_date)

    # Fetch stock data from API
    data = client.get_aggs(ticker, 1, 'day', start_date, end_date)
    
    # Convert data into a DataFrame
    df = pd.DataFrame({
        'Close': [entry.close for entry in data],
        'Volume': [entry.volume for entry in data],
        'Open': [entry.open for entry in data],
        'High': [entry.high for entry in data],
        'Low': [entry.low for entry in data], 
        'Vwap': [entry.vwap for entry in data],
        'Transactions': [entry.transactions for entry in data]
    }, index=[timestamp_to_date(entry.timestamp) for entry in data])

    # Ensure data is sorted by date
    df.sort_index(inplace=True)
    
    # Compute PROC
    df["Close_n_days_ago"] = df["Close"].shift(n)  # Closing price 'n' days ago
    df["PROC"] = ((df["Close"] - df["Close_n_days_ago"]) / df["Close_n_days_ago"]) * 100
    
    # Get the latest row with valid PROC
    latest_data = df.iloc[-1][["Close", "Close_n_days_ago", "PROC"]]
    
    # Return the final DataFrame
    result = pd.DataFrame({
        "Ticker": [ticker],
        "Current Close": [latest_data["Close"]],
        f"Close {n} Days Ago": [latest_data["Close_n_days_ago"]],
        f"PROC ({n} days)": [latest_data["PROC"]]
    })
    
    return result


In [38]:
df = stock_proc("NVDA", "2025-02-09", "2025-01-10", 10)
df

,Ticker,Current Close,Close 10 Days Ago,PROC (10 days)
0,NVDA,129.84,142.62,-8.960875


In [19]:
df.reset_index(inplace=True)  # Moves stock names from index to a column
df.rename(columns={"index": "Ticker"}, inplace=True)  # Rename the new column
df = df.loc[:, ~df.columns.duplicated()]


#Note: replaced this with fetch_macd
def compute_macd(df, short_window=12, long_window=26, signal_window=9):
    '''
    Takes in a dataframe from a certain date that was specified date and 
    calculates the MACD(Moving Average Convergence Divergence) and the signal line

    If the MACD is higher than the signal line there is a upward trend and should think of buying 
    If the MACD is lower than the signal line there is a downwards trend and should think of selling
    '''
    df = df.copy()
    df["EMA12"] = df["Close"].ewm(span=short_window, adjust=False).mean()
    df["EMA26"] = df["Close"].ewm(span=long_window, adjust=False).mean()
    df["MACD"] = df["EMA12"] - df["EMA26"]
    df["Signal_Line"] = df["MACD"].ewm(span=signal_window, adjust=False).mean()
    return df[["Ticker", "MACD", "Signal_Line"]]

macd_df = compute_macd(df)

macd_df

,Ticker,MACD,Signal_Line
0,NVDA,0.000000,0.000000
1,TQQQ,0.200946,0.040189
2,TSLA,15.015009,3.035153
3,PLTR,10.493549,4.526832
4,SRNE,6.111766,4.843819
...,...,...,...
10873,HIYS,289.532924,251.973457
10874,SYII,205.328333,242.644432
10875,AUGZ,136.144266,221.344399
10876,SUBS,79.351495,192.945818


In [ ]:
def fetch_rsi(stock_ticker, timespan="day", window=14, limit=10):
    """
    Fetches the Relative Strength Index (RSI) for a given stock ticker.
    RSI > 70 → Overbought (possible price decrease)
    RSI < 30 → Oversold (possible price increase)
    """
    url = f"https://api.polygon.io/v1/indicators/rsi/{stock_ticker}"

    params = {
        "timespan": timespan,
        "adjusted": "true",
        "window": window,
        "series_type": "close",
        "order": "desc",
        "limit": limit,
        "apiKey": key
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.json()}")
        return None

    data = response.json()

    if "results" not in data or "values" not in data["results"]:
        print("No RSI data found.")
        return None

    # Convert response to DataFrame
    rsi_data = pd.DataFrame(data["results"]["values"])
    rsi_data["timestamp"] = pd.to_datetime(rsi_data["timestamp"], unit='ms')  # Convert timestamp to datetime
    rsi_data.rename(columns={"value": "RSI"}, inplace=True)

    return rsi_data

rsi_df = fetch_rsi("AAPL", limit=10)

rsi_df

In [ ]:
def range_stock_data(start_date, end_date, stock_ticker):
    """
    Fetches historical stock data (High, Low, Close) over a date range.
    
    """
    data = client.get_aggs(
        ticker=stock_ticker,
        multiplier=1,
        timespan="day",
        from_=start_date,
        to=end_date,
        adjusted=True,
        sort="asc",
        limit=5000
    )

    df = pd.DataFrame({
        'Timestamp': [stock.timestamp for stock in data],
        'Close': [stock.close for stock in data],
        'Volume': [stock.volume for stock in data],
        'Open': [stock.open for stock in data],
        'High': [stock.high for stock in data],
        'Low': [stock.low for stock in data], 
        'Vwap': [stock.vwap for stock in data],
        'Transactions': [stock.transactions for stock in data]},
    )

    df["Timestamp"] = pd.to_datetime(df["Timestamp"], unit='ms')  # Convert timestamp to datetime
    df = df.sort_values('Timestamp', ascending=True)  # Sort data by time

    return df


In [ ]:
df = range_stock_data("2024-01-01", "2024-02-15", "AAPL")
df

In [ ]:
def calculate_williams_r(stock_data, window=14):
    """
    Calculates Williams %R for the given stock data.
    Above -20 → Sell signal
    Below -80 → Buy signal
    """
    stock_data = stock_data.sort_values("Timestamp", ascending=True)

    # Calculate Williams %R using a rolling window
    stock_data["Williams %R"] = stock_data.apply(
        lambda row: (
            (max(stock_data.loc[row.name - window + 1: row.name, "High"]) - row["Close"]) /
            (max(stock_data.loc[row.name - window + 1: row.name, "High"]) - 
             min(stock_data.loc[row.name - window + 1: row.name, "Low"]))
        ) * -100 if row.name >= window - 1 else None, axis=1
    )

    return stock_data

In [ ]:
stock_data = multiple_stock_table("2024-01-01", "2024-02-15", "AAPL")
df = calculate_williams_r(stock_data)
df.dropna()